# Modelo de Clasificacion de Twittes

In [1]:
# Bloque 1
from bs4 import BeautifulSoup
import requests

#Bloque 2
import pandas as pd
import re

#Bloque 3
from gensim.models import word2vec
import numpy as np
import gensim
from multiprocessing import cpu_count

#Bloque 4 y 5
from sklearn import datasets
from sklearn import svm
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm_notebook as tqdm       # from tqdm import tqdm
from time import sleep



In [3]:
#example  de tqdm , para medir tiempo de un bucle
for i in tqdm(range(10000)):   
    ...
    

## Parte 1 : Recoleccion de datos(ejem:Wikis) (Scraping/Crawling) 

## Leemos el dataset de Twittes Recolectados

In [2]:
data = pd.read_csv("bd_tweets.csv")

In [3]:
data.tail()

,fecha,texto
18347,2020-06-25 00:31:18,#Vizcarra solo quiere subir en las encuestas. ...
18348,2020-06-25 00:27:52,"#vizcarra ""Por que te quiero abrazar mañana, m..."
18349,2020-06-25 00:27:49,Cuando llegue las 47 horas con 50 segundo aluc...
18350,2020-06-25 00:26:18,38 mil millones de dólares en empréstitos obtu...
18351,2020-06-25 00:24:21,Las Clínicas tienen 48 horas para aceptar la o...


In [4]:
#Eliminar la columna [fecha], eliminar multiples columnnas : invoices.drop(['invoice', 'client', 'units'], axis=1)
data = data.drop(['fecha'], axis=1)

In [5]:
data.columns = ["Original"]
data.head()

,Original
0,Este fiscal está llenando la cárcel d prisione...
1,Requisitos indispensables para ser Ministro de...
2,Pedirle a la ciudadania ser PREVENIDA contra e...
3,Gracias a #Vizcarra la policía tiene harto tra...
4,Parece que si seguimos con #Vizcarra sólo crec...


## Parte 2: Pre-Procesamiento

#### Preprocesamiento Intermedio

In [6]:
diccionario = {
    "😊": "felicidad",
    "😅": "risas",
    "🤣": "risas",
    "❤️": "amor",
    "😰": "preocupacion",
    "😎": "relajacion",
    "💪": "animo",
    "😉": "broma",
    "😍": "encantado",
    "😆": "risas",
    "😄": "risas",
    "😃": "risas",
    "😇": "feliz",
    "🙃": "sonrisa",
    "😌": "tranquilidad",
    "😜": "broma",
    "🤪": "alegria",
    "😫": "impotencia",
    "😩": "infelicidad",
    "😫": "frustración",
    "🙁": "triste",
    "😢": "muy triste",
    "🤬": "maldad",
    "😤": "enojada",
    "😡": "muy enojada",
    "😠": "molesto",
    "😨": "asustado",
    "😐": "neutral",
    "🤔": "dudoso",
    "😲": "sorpresa",
    "🤥": "mentira",
    "🙄": "indiferencia",
    "🤑": "dinero",
    "🙀": "sorprendido",
    "👍": "aprobación",
    "👎": "desaprobación",
    "👏": "admiración"
    
}

def reemplazarEmoji(sentence): 
    new_sntc = ""
    for word in sentence:
        if(word in diccionario.keys()):
            new_sntc += " " + diccionario[word] + " "
        else:
            new_sntc += word
    return new_sntc

print(reemplazarEmoji("Hoy estoy 😰"))

Hoy estoy  preocupacion 


In [7]:
def eliminarTildes(texto):
    a, b = 'áéíóúüÁÉÍÓÚÜ', 'aeiouuAEIOUU'
    trans = str.maketrans(a, b)                # maketrans  de python:crea un mapeo uno a uno de un personaje para su traducción
                                                #utilizable para el método translate ()
    return texto.translate(trans)              #transalate traduce funcion de python, devuelve una cadena donde cada carácter

eliminarTildes("áéí el niño está en el avión")

'aei el niño esta en el avion'

In [8]:
def descomponerHastag(texto):
    
    if(texto[0]!="#"):
        return texto
    
    else:
        texto = texto[1:]
        
        if(texto == texto.upper()):           #COVID -> covid
            return texto.lower()
                                             
        else:                                #HolaPeru -> " Hola Peru"

            new_word = ""
            for x in texto:
                if( 65 <= ord(x) <= 90 ): # si es mayuscula     #ord, es el valor en sistema unicode.
                    new_word += " " + x
                else:
                    new_word += x
            return new_word.strip()       #strip es para elimina eliminar los espacios al inicio o al final
        
print(descomponerHastag("#YoMeQuedoEnCasa"), descomponerHastag("#100DíasDeHambreYMuerte"))

Yo Me Quedo En Casa 100 Días De Hambre Y Muerte


In [9]:
# leer la lista de stopwords
file = open("stopwords.txt","r",encoding="utf-8")
stop_words = list(line[:-1] for line in file)
file.close()

In [10]:
# cambiar las tilder

def limpiarTexto(texto):
    
    texto = re.sub('https://[./\w]+', '', texto)               # eliminar urls
    texto = reemplazarEmoji(texto)                             # reemplazar emojis
    texto = eliminarTildes(texto)                              # reemplazar las tildes
    lista = re.findall("[#@a-zA-Zñáéíóú]+", texto)             # identificar palabras en español, nos quedamos solo con palabras español
    lista = [ x for x in lista if x[0] != "@" ]                # eliminar los nombres de usuarios de twitter @Mario
    lista = [ descomponerHastag(x) for x in lista ]            # descomponer los hashtags
    texto = " ".join(list(lista))                              # concatenar la lista
    
    lista = re.findall("[#@a-zA-Zñáéíóú]+", texto)             # identificar palabras en español
    lista = list(filter(lambda x: x not in stop_words, lista)) # eliminar stopwords  # porque eso vocabulario noe es importante para nuestro analisis
    lista = filter(lambda x: len(x) > 2, lista)                # eliminar rt, q, xq, ...str>2
    texto = " ".join(list(lista))                              # concatenar la lista 
    texto = texto.strip()                                      # eliminar espacios innecesarios
    texto = texto.lower()                                      # todo a minusculas
    
    return texto

#### Aplicamos  funcion de Pre-Procesamiento

In [11]:
data["limpio"] = data["Original"].apply(limpiarTexto)
data.head()

,Original,limpio
0,Este fiscal está llenando la cárcel d prisione...,este fiscal llenando carcel prisiones preventi...
1,Requisitos indispensables para ser Ministro de...,requisitos indispensables ministro vizcarra te...
2,Pedirle a la ciudadania ser PREVENIDA contra e...,pedirle ciudadania prevenida coronavirus pais ...
3,Gracias a #Vizcarra la policía tiene harto tra...,gracias vizcarra policia harto
4,Parece que si seguimos con #Vizcarra sólo crec...,parece seguimos vizcarra crecera gasto estado ...


## Parte 3: Extraccion de Vectores Caracteristicos

### Metodo 1: Tecnica de TF - IDF

#### Proceso previo si trabajaremos con TF1-IDF

In [12]:
#TF1-IDF: Nuestro top_n = serian todo las palabras del vector universo sin repeticion
lista_tweets = list(data["limpio"])
strg = " ".join(lista_tweets)

#Numero de palabras en total del vector universo con repetidas
len(strg.split())

#Numero de palabras en total del vector universo sin repetir (Este seria el top_n si trabajamos con "TF1")
len(strg.split())
len(set(strg.split()))

27811

#### Si trabajaremos con TF2-IDF, empezamos apartir de esta funcion 

In [19]:
#corpus = lista de strings: TF2-IDF
def tf_idf(corpus, top_n = 1000, idf =True):  #1000 top_n:tamaño(salida) de los vectores caracteristicos(podemos variar)
    
    corpus_org =  " ".join(corpus).split()
    corpus_unique = list(set(corpus_org))
    
    #ordenar por frecuencia de palabras y obtener los top (lista_de_tuplas)
    d = {}
    for word in tqdm(corpus_unique):
        d[word] = corpus_org.count(word)
        
    d_sorted = sorted(d.items(), key=lambda kv: kv[1], reverse=True)[:top_n]  # ordena el diccionario por valor no por clave(Key), reverse(decreciente)
                                                                            #d_sorted: devuelve una lista de tuplas(palabra,conteo)
    lista_tops = [word[0] for word in d_sorted]
    
    FV = []            #Seria mi lista de vectores caracteristicos (lista de numpy_arrays)
    
    for cad in corpus:
        
        #inicializando Vector U
        U = np.zeros(top_n)
        for w in cad.split():
            
            if w in lista_tops:
                
                ind = lista_tops.index(w)    #  index: indices de mis valores
                U[ind] = U[ind] + 1
        
        FV.append(U)
        
    if not idf:  # solo seria TF
        return np.vstack(FV)  # vstack: almacena en una matriz array una (lista de listas), se apila en formato numpy
    
    else:   # deiseñmos el IDF, en caso hayamos sugerido idf=True
        
        idf = np.ones(top_n)    # seria mi lista de arrays que contienen los (idfs de cada palabra) 
        Num_docs = len(corpus)  #Numero de documentos que ingresan  en total
        
        for word in lista_tops:  # lista_tops: son los documentos que estan el top, pues se repiten mas, solo te quedas con las palbras , omites el conteo
            
                count = 0
                for documento in corpus:
                    
                    if word in documento:
                        count = count + 1  #contador ira aumentando de 1 en 1
                        
                ind = lista_tops.index(word)  #genenarmos el indice de la palabra, para ver en que posicion esta la palabra
                
                idf[ind] = np.log( Num_docs / count )# idf = Log(Numero_documentos_total / Nt,count = Numero_Documentos_q_contienen_palabra_t)   
                    
        return  np.vstack(FV)*idf   # multiplicacion de elmento a elmento 
            

In [20]:
# Corpus limpio 
Lista_twittes_limpia = data["limpio"].values.tolist()  #data["# texto_limpio"], data["Limpio"]

In [42]:
#Lista_twittes_limpia      #es extenso, evitar correr , se puede lagear la maquina

#### Escoger si queremos Vectores Carateristicos con TF, o TF-IDF

In [21]:
#TF
FV_TF = tf_idf(Lista_twittes_limpia, top_n=1000, idf=False)
data["FV_TF_1000"] = FV_TF.tolist()
data.head()

,Original,limpio,FV_TF_1000
0,Este fiscal está llenando la cárcel d prisione...,este fiscal llenando carcel prisiones preventi...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,Requisitos indispensables para ser Ministro de...,requisitos indispensables ministro vizcarra te...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,Pedirle a la ciudadania ser PREVENIDA contra e...,pedirle ciudadania prevenida coronavirus pais ...,"[0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
3,Gracias a #Vizcarra la policía tiene harto tra...,gracias vizcarra policia harto,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,Parece que si seguimos con #Vizcarra sólo crec...,parece seguimos vizcarra crecera gasto estado ...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [48]:
#TF - IDF
FV_TF_idf = tf_idf(Lista_twittes_limpia, top_n=1000, idf=True)
data["FV_TF_idf_1000"] = FV_TF_idf.tolist()
data.head()

,Original,limpio,FV_TF_idf_1000
0,"El deporte es una actividad reglamentada, norm...",deporte actividad reglamentada normalmente car...,"[2.4738180456025622, 0.8650755403244127, 0.523..."
1,El Diccionario de la lengua española (oficialm...,diccionario lengua española oficialmente desde...,"[0.5093154799769981, 0.43253777016220635, 0.52..."
2,La diversión es el uso del tiempo de una maner...,diversión tiempo manera planeada para refresco...,"[0.0, 0.05406722127027579, 0.10471095408903076..."
3,"Los deportes electrónicos, también conocidos c...",deportes electrónicos también conocidos como e...,"[1.3096683770837094, 0.7569410977838611, 0.628..."
4,Los medios de comunicación de masas (en inglés...,medios comunicación masas inglés mass media me...,"[3.0558928798619887, 1.567949416837998, 1.8847..."


In [58]:
data.columns

Index(['Original', 'limpio', 'FV_TF_1000'], dtype='object')

### Metodo 2: Tecnica de Word2Vec

In [59]:
#Exportar la lista de wikis en string(cadena) en .txt (utf-8)
f= open("Twittes_limpias.txt","w",encoding = "utf-8")
f.writelines(Lista_twittes_limpia)
f.close()

In [60]:
#traemos el texto guaradado en utf'8, para poder entrenar nuestro modelo
sentences = word2vec.Text8Corpus("Twittes_limpias.txt")

In [62]:
#Creamos el modelo word2vec(basado en red neuronal)
model = word2vec.Word2Vec(sentences,size=50,min_count=2, workers=cpu_count())

In [63]:
model.wv.vocab

{'este': <gensim.models.keyedvectors.Vocab at 0x20f41b0fc50>,
 'fiscal': <gensim.models.keyedvectors.Vocab at 0x20f41b0f240>,
 'llenando': <gensim.models.keyedvectors.Vocab at 0x20f41b0fdd8>,
 'carcel': <gensim.models.keyedvectors.Vocab at 0x20f41b0f5f8>,
 'prisiones': <gensim.models.keyedvectors.Vocab at 0x20f41b0f080>,
 'preventivas': <gensim.models.keyedvectors.Vocab at 0x20f41b0f320>,
 'acusa': <gensim.models.keyedvectors.Vocab at 0x20f41b0fe80>,
 'protegidos': <gensim.models.keyedvectors.Vocab at 0x20f41b0f9e8>,
 'delincuentes': <gensim.models.keyedvectors.Vocab at 0x20f41b0fef0>,
 'saqueado': <gensim.models.keyedvectors.Vocab at 0x20f41b0f0f0>,
 'pais': <gensim.models.keyedvectors.Vocab at 0x20f41b0f198>,
 'puro': <gensim.models.keyedvectors.Vocab at 0x20f41b0f208>,
 'show': <gensim.models.keyedvectors.Vocab at 0x20f41b0f390>,
 'odebrecht': <gensim.models.keyedvectors.Vocab at 0x20f41b0f3c8>,
 'robando': <gensim.models.keyedvectors.Vocab at 0x20f41b0f4a8>,
 'facturando': <gensim.

In [66]:
#Probamos el modelo buscando palabaras similares a la ingresada
model.wv.most_similar(positive=["vizcarra"],topn=10)

[('incapaz', 0.9820099472999573),
 ('infeliz', 0.9808326959609985),
 ('cumpleanos', 0.9775086045265198),
 ('oppa', 0.9740936160087585),
 ('problema', 0.972922682762146),
 ('destruyo', 0.9721062779426575),
 ('vete', 0.9718038439750671),
 ('entienda', 0.9710962176322937),
 ('hundiendo', 0.9695389866828918),
 ('desearle', 0.9661450386047363)]

In [68]:
#Probamos el modelo buscando palabaras similares a la ingresada
model.wv.most_similar_cosmul('corona')

[('challenge', 0.9941051006317139),
 ('godoy', 0.989782452583313),
 ('virus', 0.9876828789710999),
 ('combatimos', 0.9791717529296875),
 ('peruponen', 0.9727088212966919),
 ('loteria', 0.9708677530288696),
 ('covidexcelente', 0.9667409658432007),
 ('outbreak', 0.9639636874198914),
 ('update', 0.9632783532142639),
 ('propagar', 0.9610130190849304)]

In [69]:
def promediarVectores(texto,model):
    
    k = 0
    s = np.zeros(model.vector_size)  # vector_size:tamaño de salida de modelo (n°neuronas)(ejem:doc1:salida(n neuronas))
    
    for palabra in texto.split():
        
        if palabra in model.wv.vocab:
            s = s + model[palabra]
            k = k + 1
            
    if k == 0:
        return s
    else:
        return s/k 
    

In [70]:
#L_Vectores = [promediarVectores(wiki_limpia) for wiki_limpia in Lista_texto_wikis]
data["vectores_w2vec"] = data["limpio"].apply(promediarVectores, args=(model,))
data.head()

,Original,limpio,FV_TF_1000,vectores_w2vec
0,Este fiscal está llenando la cárcel d prisione...,este fiscal llenando carcel prisiones preventi...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.32757393368764925, 0.17691281859419847, -0..."
1,Requisitos indispensables para ser Ministro de...,requisitos indispensables ministro vizcarra te...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.3063852957636118, 0.1582631541788578, -0.3..."
2,Pedirle a la ciudadania ser PREVENIDA contra e...,pedirle ciudadania prevenida coronavirus pais ...,"[0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.43121133257141886, 0.3406322327825953, -0...."
3,Gracias a #Vizcarra la policía tiene harto tra...,gracias vizcarra policia harto,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.6236122287809849, 0.3811798980459571, -0.6..."
4,Parece que si seguimos con #Vizcarra sólo crec...,parece seguimos vizcarra crecera gasto estado ...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.3541022029650562, 0.20808624344713547, -0...."


## Parte 4 : Implementación de un proceso de clasificación  

In [76]:
# Etiquetamos nuestra data: primeros 100 = "Deporte"  y los demas "Religion"
etiquetas = [0]*(int(len(Lista_twittes_limpia)/2)) + [1]*(int(len(Lista_twittes_limpia)/2))  #Los primeros 70: Deporte , los restantes : Religion o (Lista_texto_wikis/2)
data["Labels"] = etiquetas
data.head()

,Original,limpio,FV_TF_1000,vectores_w2vec,Labels
0,Este fiscal está llenando la cárcel d prisione...,este fiscal llenando carcel prisiones preventi...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.32757393368764925, 0.17691281859419847, -0...",0
1,Requisitos indispensables para ser Ministro de...,requisitos indispensables ministro vizcarra te...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.3063852957636118, 0.1582631541788578, -0.3...",0
2,Pedirle a la ciudadania ser PREVENIDA contra e...,pedirle ciudadania prevenida coronavirus pais ...,"[0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.43121133257141886, 0.3406322327825953, -0....",0
3,Gracias a #Vizcarra la policía tiene harto tra...,gracias vizcarra policia harto,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.6236122287809849, 0.3811798980459571, -0.6...",0
4,Parece que si seguimos con #Vizcarra sólo crec...,parece seguimos vizcarra crecera gasto estado ...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.3541022029650562, 0.20808624344713547, -0....",0


In [77]:
def clasificar_FV(data, fv, C=1, g=0.1, k="rbf"):
    
    if fv == "tf":
        col = "FV_TF_1000"
    elif fv == "tf_idf":
        col = "FV_TF_idf_1000"
    elif fv == "w2v":
        col = "vectores_w2vec"
    
    x = np.stack(data[col].values)
    y = data["Labels"].values
    
    X_train,X_test,Y_train,Y_test = train_test_split(x,y,test_size=0.2,random_state=10)
    
    print("Dataset: \n")
    print("Train = "+str(X_train.shape[0]))
    print("Test = "+str(X_test.shape[0]))
    
    modelol = svm.SVC(C=C, gamma=g, kernel=k)
    modelol.fit(X_train,Y_train)      #entrenamiento
    
    Y_pred = modelol.predict(X_test)   #prediccion
    print("\nAccuracy = " + str(accuracy_score(Y_test, Y_pred)))
          
    print("\nClassification Report:\n\n"+classification_report(Y_test,Y_pred))
          
             

In [78]:
clasificar_FV(data,"w2v")

Dataset: 

Train = 14681
Test = 3671

Accuracy = 0.6900027240533915

Classification Report:

              precision    recall  f1-score   support

           0       0.71      0.64      0.67      1838
           1       0.67      0.74      0.71      1833

    accuracy                           0.69      3671
   macro avg       0.69      0.69      0.69      3671
weighted avg       0.69      0.69      0.69      3671



## Parte 5 : Optimizar Clasificador usando Croos Validation y Determinar mejor Descriptor

In [79]:
col = "FV_TF_1000"
#col = "FV_TF_idf_1000"
#col = "vectores_w2vec"

X = np.stack(data[col].values)
Y = data["Labels"].values

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=10)

In [80]:
params = {'kernel':('linear','rbf'),'C':[1,10,100],'gamma':[0.1,0.001,0.0001,0.00001]}

In [81]:
clf = svm.SVC()

In [82]:
clf = GridSearchCV(clf,params,cv=5,verbose=1,n_jobs=-1)

In [ ]:
clf.fit(X_train,Y_train)     #entrenamiento

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [102]:
#Mejores parametros
clf.best_params_

{'C': 100, 'gamma': 1e-05, 'kernel': 'rbf'}

In [ ]:
#Clasificando con parametros optimos obtneidos con Croos Validation

In [103]:
clasificar_FV(data, "tf_idf", C=100, g=0.00001, k="rbf")

Dataset: 

Train = 136
Test = 35

Accuracy = 0.8

Classification Report:

              precision    recall  f1-score   support

           0       0.86      0.50      0.63        12
           1       0.79      0.96      0.86        23

    accuracy                           0.80        35
   macro avg       0.82      0.73      0.75        35
weighted avg       0.81      0.80      0.78        35



In [104]:
clasificar_FV(data, "w2v", C=100, g=0.00001, k="rbf")

Dataset: 

Train = 136
Test = 35

Accuracy = 0.6571428571428571

Classification Report:

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        12
           1       0.66      1.00      0.79        23

    accuracy                           0.66        35
   macro avg       0.33      0.50      0.40        35
weighted avg       0.43      0.66      0.52        35

